In [1]:
"""
Home Win modelling

create predictions and for the hockey data

input:
output:
"""

credential_keys = '/Users/antoinetl/Documents/nhl_prediction/nhl_prediction/account_keys/hockey-prediction-qc-9c75aa8a78f9.json'

In [2]:
# Import the important packages
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

pd.set_option('display.max_columns', None)

In [3]:
# Load the data

# Connexion GBQ
import pandas_gbq
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file(
    credential_keys,
    )
pandas_gbq.context.credentials = credentials
# https://pandas-gbq.readthedocs.io/en/latest/intro.html

# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "hockey-prediction-qc"


sql = """
SELECT *
from `hockey_prediction_qc.home_win_df`
"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████| 1024/1024 [00:00<00:00, 2227.11rows/s]


In [33]:
#df = df.set_index('gameID')
df.sort_values('date').tail()

,gameID,date,teams_home_team_id,teams_away_team_id,Home_win,ppg,ppg_away,ppg_diff,ppg_last_10,ppg_last_10_away,ppg_last_10_diff
941,2017021259,2018-04-07 00:00:00+00:00,10,8,1.0,1.864198,1.234568,0.629630,1.8,1.0,0.8
873,2017021260,2018-04-07 00:00:00+00:00,17,2,0.0,1.259259,1.382716,-0.123457,1.3,1.2,0.1
186,2017021263,2018-04-07 00:00:00+00:00,12,14,1.0,1.432099,2.049383,-0.617284,1.5,1.8,-0.3
439,2017021267,2018-04-07 00:00:00+00:00,53,24,0.0,1.222222,1.753086,-0.530864,1.9,2.5,-0.6
977,2017020616,2018-04-08 00:00:00+00:00,6,13,0.0,2.000000,1.691358,0.308642,1.8,1.9,-0.1


In [5]:
df.describe(include='all')

,gameID,date,teams_home_team_id,teams_away_team_id,Home_win,ppg,ppg_away,ppg_diff,ppg_last_10,ppg_last_10_away,ppg_last_10_diff
count,1.024000e+03,1024,1024.000000,1024.000000,1024.000000,1024.000000,1024.000000,1024.000000,1024.000000,1024.000000,1024.000000
unique,NaN,145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,2017-12-23 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,2017-11-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2018-04-08 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2.017021e+09,NaN,18.962891,18.910156,0.571289,1.619734,1.618797,0.000937,1.630176,1.609375,0.020801
std,2.959936e+02,NaN,13.921459,13.857596,0.495134,0.298920,0.304770,0.440457,0.475250,0.497039,0.710449
min,2.017020e+09,NaN,1.000000,1.000000,0.000000,0.500000,0.450000,-1.478788,0.100000,0.400000,-1.900000
25%,2.017021e+09,NaN,8.000000,8.000000,0.000000,1.426667,1.414150,-0.306935,1.300000,1.300000,-0.500000


In [6]:
y = df['Home_win']

#X_dum1 = pd.get_dummies(mod_input['season'], prefix='season')
#X_dum2 = pd.get_dummies(mod_input['teams.away.team.name'], prefix='team.away')
#X_dum3 = pd.get_dummies(mod_input['teams.home.team.name'], prefix='team.home')

#X = mod_input.drop(columns=['gameType','game.homewin', 'Unnamed: 0','season','teams.away.team.name','teams.home.team.name','teams.home.leagueRecord.gameplayed','teams.away.leagueRecord.gameplayed'])
X = df.drop(columns=['Home_win','gameID', 'teams_home_team_id', 'teams_away_team_id', 'date'])
#X = pd.concat([X, X_dum1, X_dum2, X_dum3], axis=1)

In [7]:
X

,ppg,ppg_away,ppg_diff,ppg_last_10,ppg_last_10_away,ppg_last_10_diff
0,1.210526,1.776316,-0.565789,0.3,2.0,-1.7
1,1.456790,1.975309,-0.518519,0.3,2.0,-1.7
2,1.236111,1.671233,-0.435122,0.1,1.0,-0.9
3,2.104478,1.617647,0.486831,3.0,1.2,1.8
4,1.926829,1.704545,0.222284,2.6,1.9,0.7
...,...,...,...,...,...,...
1019,1.619048,1.736842,-0.117794,1.8,1.8,0.0
1020,1.483871,1.666667,-0.182796,1.8,1.8,0.0
1021,2.111111,1.759259,0.351852,1.8,1.8,0.0
1022,1.733333,1.766667,-0.033333,1.8,1.8,0.0


In [8]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)  
X_train = scaler.transform(X_train) # only compute mean and std here
X_test = scaler.transform(X_test)

In [10]:
#Import svm model
#from sklearn import svm

#Create a svm Classifier
#clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
#clf.fit(X_train, y_train)

#Predict the response for test dataset
#y_pred = clf.predict(X_test)

In [11]:
clf=RandomForestClassifier(n_estimators=1000)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [12]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.model_selection import GridSearchCV
# import numpy as np

# parameters = {
#     "loss":["deviance"],
#     "learning_rate": [0.01, 0.05, 0.1, 0.2],
#     "min_samples_split": np.linspace(0.1, 0.5, 10),
#     "min_samples_leaf": np.linspace(0.1, 0.5, 10),
#     "max_depth":[3,8],
#     "max_features":["sqrt"],
#     "subsample":[0.5, 0.8, 0.9, 1.0],
#     "n_estimators":[10]
#     }

# clf = GridSearchCV(GradientBoostingClassifier(), 
#                    parameters, 
#                    cv=5, 
#                    n_jobs=-1, 
#                    scoring='accuracy')

# clf.fit(X_train, y_train)
# print(clf.score(X_train, y_train))
# print(clf.best_params_)
# y_pred=clf.predict(X_test)

In [13]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.525974025974026


In [14]:
#feature_importances = pd.DataFrame(clf.feature_importances_,
#                                   index = X_train.columns,
#                                   columns=['importance']).sort_values('importance', ascending=False)

In [15]:
#print(feature_importances.head(10))

In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.41      0.45      0.43       121
         1.0       0.62      0.58      0.60       187

    accuracy                           0.53       308
   macro avg       0.51      0.51      0.51       308
weighted avg       0.53      0.53      0.53       308



In [17]:
y_pred=clf.predict(X)

In [18]:
y_pred.shape

(1024,)

In [19]:
df.shape

(1024, 11)

In [20]:
type(y_pred)

numpy.ndarray

In [21]:
type(df)

pandas.core.frame.DataFrame

In [22]:
y_pred_df = pd.DataFrame(data=y_pred.flatten())

In [23]:
y_pred_df.head()
y_pred_df.columns = ['Pred']

In [24]:
y_pred_df.shape

(1024, 1)

In [25]:
df.head()

,gameID,date,teams_home_team_id,teams_away_team_id,Home_win,ppg,ppg_away,ppg_diff,ppg_last_10,ppg_last_10_away,ppg_last_10_diff
0,2017021179,2018-03-27 00:00:00+00:00,17,5,1.0,1.210526,1.776316,-0.565789,0.3,2.0,-1.7
1,2017021268,2018-04-07 00:00:00+00:00,20,54,1.0,1.456790,1.975309,-0.518519,0.3,2.0,-1.7
2,2017021126,2018-03-20 00:00:00+00:00,17,4,1.0,1.236111,1.671233,-0.435122,0.1,1.0,-0.9
3,2017021057,2018-03-10 00:00:00+00:00,18,1,0.0,2.104478,1.617647,0.486831,3.0,1.2,1.8
4,2017020683,2018-01-15 00:00:00+00:00,6,25,0.0,1.926829,1.704545,0.222284,2.6,1.9,0.7


In [26]:
predictions = pd.concat([df, y_pred_df], axis=1)

In [27]:
#predictions.columns[-1] = 'Antoine'
#gapminder.rename(columns={0}) = 'Antoine'

y_pred_df.head()

,Pred
0,1.0
1,1.0
2,1.0
3,0.0
4,0.0


In [28]:
final_predictions_gcp = predictions[['date', 'teams_home_team_id', 'teams_away_team_id', 'Pred', 'Home_win']]

In [30]:
# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "hockey-prediction-qc"

# TODO: Set table_id to the full destination table ID (including the dataset ID).
table_id = 'hockey_prediction_qc.final_pred'

pandas_gbq.to_gbq(final_predictions_gcp, table_id, project_id=project_id, if_exists='replace')

1it [00:05,  5.01s/it]


In [31]:
final_predictions_gcp.head()

,date,teams_home_team_id,teams_away_team_id,Pred,Home_win
0,2018-03-27 00:00:00+00:00,17,5,1.0,1.0
1,2018-04-07 00:00:00+00:00,20,54,1.0,1.0
2,2018-03-20 00:00:00+00:00,17,4,1.0,1.0
3,2018-03-10 00:00:00+00:00,18,1,0.0,0.0
4,2018-01-15 00:00:00+00:00,6,25,0.0,0.0
